1.INSTALLATION


In [1]:

%pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.htmlNote: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/3.1 GB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 GB 1.6 MB/s eta 0:32:41
     ---------------------------------------- 0.0/3.1 GB 1.5 MB/s eta 0:34:16
     ---------------------------------------- 0.0/3.1 GB 1.4 MB/s eta 0:36:35
     ---------------------------------------- 0.0/3.1 GB 2.6 MB/s eta 0:19:20
     ---------------------------------------- 0.0/3.1 GB 3.0 MB/s eta 0:17:13
     ---------------------------------------- 0.0/3.1 GB 2.9 MB/s eta 0:17:31
     ---------------------------------------- 0.0/3.1 GB 2.7 MB/s eta 0:18:45
     ---------------------------------------- 0.0/3.1 GB 2.7 MB/s eta 0:18:39
     ---------------------------------------- 0.0/3.1 GB 2.7 MB/s eta 0:18:39
     ---------------------------------------- 0.0/3.1 GB 2.9 MB/s eta 0:17:19
     ---------

In [2]:
%pip install transformers requests beautifulsoup4 pandas numpy

  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     -----------------

2.INTIALIZE MODEL

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

d:\system\anaconda3\envs\BERT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

CALCULATION

In [5]:
# return_tensors = pt  #pytorch

tokens= tokenizer.encode("It was good but codve been better. Great", return_tensors='pt')

In [9]:
print(tokens[0])
tokenizer.decode(tokens[0])


tensor([  101, 10197, 10140, 12050, 10502, 60451, 10598, 10662, 16197,   119,
        11838,   102])


'[CLS] it was good but codve been better. great [SEP]'

In [7]:
result=model(tokens)

In [10]:
# result
print(result.logits)
int(torch.argmax(result.logits))+1
# values from 1-5 rates the comment


tensor([[-2.3461, -1.0148,  1.1667,  1.5955,  0.4477]],
       grad_fn=<AddmmBackward>)


4

REVIEW COLLECTION

In [13]:
r = requests.get('https://www.yelp.com/biz/caliente-bistro-kitchen-san-francisco?osq=Mexican')
# print(r.text)
soup = BeautifulSoup(r.text,"html.parser")
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
# print(results)
# print(results[0])
reviews = [result.text for result in results]
# print(reviews)



['Great food period. Great service. Great chips and salsa! Enchiladas were meant and flavorful and not over sauced', 'I went in on a cold and rainy day to order a takeout burrito and I saw a sign they had Pozole as a special. It was beyond scrumptious!!With the cilantro, limes,etc on the side and my stomach and heart was more than satisfied.My 3rd visit I ate in, ordered enchiladas, was greeted warmly and again my meal in no way disappointed. The chicken and the seasonings were sublime and the black beans were prepared perfectly. The on-screen salsa performance was a perfect compliment. I am sure I will be back.', 'Do NOT come here if you want authentic Mexican food. I hate to say it but the food was incredibly bland. I ordered some "enchiladas" and was disappointed. It lacked flavor and was not drowning in enchilada sauce like you normally see at other restaurants.At least the staff were nice.', "Really delicious and decently priced! When you're seated you're given FRESH chips and sal

LOAD REVIEWS INTO DATAFRAME


In [14]:
import pandas as  pd
import numpy as np  

In [20]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
# print(df.head)
# print(df.tail)
print(df['review'].iloc[0])

Great food period. Great service. Great chips and salsa! Enchiladas were meant and flavorful and not over sauced


In [18]:
def sentiment_score(review):
    tokens= tokenizer.encode(review, return_tensors='pt')
    result=model(tokens)
    return int(torch.argmax(result.logits))+1

In [21]:
print(sentiment_score(df['review'].iloc[0]))

5


In [22]:
df['sentiment'] = df['review'].apply(lambda x:  sentiment_score(x[:512]))

In [23]:
print(df)

                                               review  sentiment
0   Great food period. Great service. Great chips ...          5
1   I went in on a cold and rainy day to order a t...          5
2   Do NOT come here if you want authentic Mexican...          2
3   Really delicious and decently priced! When you...          4
4   We had to pick up an item in the area, I decid...          5
5   I have 2 pictures. Before and after pork ribs....          5
6   Update review: love their happy hour. Their $2...          5
7   Another review for one of my favorite lunch pl...          5
8   After a long hiatus, we returned. Still great!...          5
9   Made our 2nd trip here before commenting. Had ...          5
10  Absolutely five stars for food and service. Th...          5
11  Will be coming here everyday for happy hour!! ...          5
12  I'm not sure if they taste super "traditional"...          4
13  Surprised to find a mexican restaurant in this...          5
